<a href="https://colab.research.google.com/github/ZaneZaiontz/YosAI/blob/main/YosAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO:**


*   List item
*   List item



Imports

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from keras.preprocessing import sequence
import keras
import numpy as np
import os
import sys
import csv
import random as rand
import pandas as pd

Open dataset


In [2]:
# Read, then decode for py2 compat.
text = open('./zhaoClean.txt', 'r').read()#.decode(encoding='utf-8').lower()
# num chars in file
print ('Length of text: {} characters'.format(len(text)))

Length of text: 256636 characters


Encoding


In [3]:
vocab1 = {}
currCode = 1
def wordEncode(text):
  global currCode

  words = text.lower().split(' ')
  #t1=''.join(words)
  #words = t1.split(' ')
  
  #print(words[20])


  encoding = []

  for word in words:
    if word in vocab1:
      code = vocab1[word]
      encoding.append(code)
    else:
      vocab1[word] = currCode
      encoding.append(currCode)
      currCode += 1
  
  return encoding

#text = "this is a test to see if this test will work is is test a a"
encoding = wordEncode(text)
print(encoding[90:1000])
print(vocab1['why'])
print(len(vocab1))

[79, 80, 81, 82, 83, 84, 85, 86, 87, 3, 88, 89, 90, 6, 91, 92, 93, 3, 94, 95, 96, 97, 98, 6, 99, 100, 35, 101, 102, 103, 104, 105, 106, 107, 108, 109, 35, 110, 111, 112, 113, 114, 6, 115, 116, 117, 94, 118, 119, 120, 121, 122, 6, 123, 124, 1, 125, 126, 127, 128, 129, 130, 126, 6, 131, 6, 132, 3, 133, 3, 134, 135, 136, 137, 3, 138, 47, 139, 140, 6, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 3, 1, 151, 152, 153, 154, 155, 3, 156, 157, 158, 6, 159, 160, 161, 149, 1, 162, 163, 164, 165, 47, 6, 166, 167, 168, 169, 170, 6, 171, 3, 6, 172, 173, 174, 175, 176, 177, 178, 69, 179, 180, 3, 6, 181, 182, 183, 184, 47, 6, 185, 186, 47, 187, 188, 189, 190, 191, 192, 1, 193, 194, 195, 196, 197, 198, 199, 200, 3, 201, 92, 163, 202, 203, 149, 204, 205, 206, 207, 35, 208, 209, 6, 210, 211, 1, 212, 213, 214, 215, 3, 216, 217, 218, 219, 6, 220, 221, 222, 223, 224, 225, 226, 204, 227, 1, 228, 1, 229, 230, 231, 232, 233, 234, 235, 69, 236, 237, 238, 239, 32, 240, 241, 153, 242, 243, 244, 245, 6, 246, 

Step 3

In [4]:
seq_length = 16
examples_per_epoch = len(text)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(encoding)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Step 4

In [5]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

Build Model/Shuffle Data

In [6]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab1)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 2048

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           4028160   
_________________________________________________________________
lstm (LSTM)                  (64, None, 2048)          18882560  
_________________________________________________________________
dense (Dense)                (64, None, 15735)         32241015  
Total params: 55,151,735
Trainable params: 55,151,735
Non-trainable params: 0
_________________________________________________________________


Compile/Loss Function


In [7]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

Checkpoints/Fitting

In [8]:
checkpoint_dir = './trainCKPT'
checkpoint_prefix = os.path.join(checkpoint_dir, "CKPT-{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)
history = model.fit(data, epochs=40, callbacks=[checkpoint_callback])

Epoch 1/40
29/29 [==============================] - 14s 120ms/step - loss: 9.3425
Epoch 2/40
29/29 [==============================] - 4s 117ms/step - loss: 8.4853
Epoch 3/40
29/29 [==============================] - 4s 119ms/step - loss: 8.1606
Epoch 4/40
29/29 [==============================] - 4s 118ms/step - loss: 7.9588
Epoch 5/40
29/29 [==============================] - 4s 118ms/step - loss: 7.8503
Epoch 6/40
29/29 [==============================] - 4s 120ms/step - loss: 7.6840
Epoch 7/40
29/29 [==============================] - 4s 120ms/step - loss: 7.4921
Epoch 8/40
29/29 [==============================] - 4s 122ms/step - loss: 7.3028
Epoch 9/40
29/29 [==============================] - 4s 121ms/step - loss: 7.0093
Epoch 10/40
29/29 [==============================] - 4s 122ms/step - loss: 6.6363
Epoch 11/40
29/29 [==============================] - 4s 123ms/step - loss: 6.2183
Epoch 12/40
29/29 [==============================] - 4s 125ms/step - loss: 5.7481
Epoch 13/40
29/29 [=====

Build

In [9]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

Output

In [19]:
def generate_text(model):

  start = rand.randint(1, 15735) # TODO: Maybe syllable or word
  startWord = get_key(start)
  # Vectorizing start char
  input_eval = [vocab1[get_key(start)]]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store results
  txtGen = []

  model.reset_states()
  while True:
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      txtGen.append(get_key(predicted_id))
      txtGen.append(' ')
      txtGen
      if syllable_count(''.join(txtGen)) > 17:
        break
  return (''.join(txtGen))

def get_key(val):
  for key, value in vocab1.items():
    if val == value:
      return key

def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def toHaiku(arr):
  i = 0
  firCheck = False
  secCheck = False
  retArr = []
  for word in arr:
    if word == '\n':
      continue

    i+=syllable_count(word)
    retArr.append(word)
    if i > 12 and not secCheck:
      retArr.append('\n')
      secCheck = True
    if i > 5 and not firCheck:
      retArr.append('\n')
      firCheck = True


  return ' '.join(retArr)


rawOut = generate_text(model)

rawOut = rawOut.split('\n')
rawOut = ' '.join(rawOut)

rawArr = rawOut.split(' ')
del rawArr[-1]
printST = toHaiku(rawArr)

print(printST)



field empty bunch machine 
 which news from - chat light uphill 
 spring distant echo across


**Output**
